In [1]:
import pandas as pd

In [54]:
d = {'col1': [1, 1, 2, 2, 3, 3], 'col2': [3, 4, 3, 5, 6 , 6], 'col3':['median','median',
                                                                      'sum' ,'sum',
                                                                     'median','median']}

df = pd.DataFrame(data=d)

In [56]:
ag = df.groupby(['col1'])['col3'].unique().apply(lambda x: ''.join(x)).values

In [57]:
ag

array(['median', 'sum', 'median'], dtype=object)

In [58]:
np.diag(df.groupby(['col1', 'col3']).agg({'col2' : ag}).values)

array([3.5, 8. , 6. ])

In [59]:
df.groupby(['col1', 'col3']).agg({'col2' : ag})

col2           
            median sum median
col1 col3                    
1    median    3.5   7    3.5
2    sum       4.0   8    4.0
3    median    6.0  12    6.0

In [74]:
df.groupby(['col1', 'col3'])['col2'].mean().reset_index()

,col1,col3,col2
0,1,median,3.5
1,2,sum,4.0
2,3,median,6.0


In [65]:
exp = np.array([2,2 ,13])

In [68]:
exp[exp==2] = np.array([2,3])

In [69]:
exp

array([ 2,  3, 13])

In [40]:
import numpy as np
import pandas as pd


def agg_comp_price(X: pd.DataFrame) -> pd.DataFrame:
    '''pipline for price'''
    x = X.copy()
    if (np.unique(x['agg']) != 'rnk').all():
        x_rnk = x[x['agg'] == 'rnk']
        x = x[x['agg'] != 'rnk']
        agreg = x.groupby(['sku'])['agg'].unique().apply(
            lambda x: ''.join(x)).values
        agreg[agreg == 'med'] = 'median'
        agreg[agreg == 'avg'] = 'mean'
        x_rnk = x_rnk[x_rnk['agg'] == 'rnk'].\
            groupby(['sku', 'agg', 'base_price'])['rank', 'comp_price'].min().\
            reset_index()

        prices = np.diag(x.groupby(['sku', 'agg']).agg(
            {'comp_price': agreg}).values)

        data = x.groupby(['sku', 'agg'])['base_price'].mean().reset_index()

        data['comp_price'] = prices
        data = pd.concat([data, x_rnk.drop('rank', axis=1)], axis=0).sort_values(by='sku').\
            reset_index(drop=True)
        data['new_price'] = np.where(abs(((100 / data['base_price']) * data['comp_price']) - 100) >= 20,
                                     data['base_price'], data['comp_price'])
        data['new_price'].fillna(data['base_price'], inplace=True)

        return data

    elif (np.unique(x['agg']) == 'rnk').all():
        x['rank'].fillna(-1, inplace = True)
        data = x.\
            groupby(['sku', 'agg', 'base_price'])['rank', 'comp_price'].min().\
            reset_index()

        data = data.drop('rank', axis=1).reset_index(drop = True)
        data['new_price'] = np.where(abs(((100 / data['base_price']) * data['comp_price']) - 100) >= 20,
                                     data['base_price'], data['comp_price'])
        data['new_price'].fillna(data['base_price'], inplace=True)
        return data

    else:
        data = x.groupby(['sku', 'agg', 'base_price'])['rank', 'comp_price'].min().\
            reset_index()

        data = data.drop('rank', axis=1)
        data['new_price'] = np.where(abs(((100 / data['base_price']) * data['comp_price']) - 100) >= 20,
                                     data['base_price'], data['comp_price'])
        data['new_price'].fillna(data['base_price'], inplace=True)
        return data

In [8]:
df = pd.DataFrame({'sku': [0, 1, 1, 2, 2, 3, 3, 4, 5, 5, 5, 6, 6, 6, 7, 8, 9],
                   'agg': ['max', 'med', 'med', 'avg', 'avg', 'rnk', 'rnk', 'max', 'max', 'max', 'max', 'min',  'min',  'min', 'med', 'rnk', 'rnk'],
                   'rank': [-1, 0, 1, 0, 1, 0, 2, 0, 0, 1, 2, 0, 1, 2, 0, -1, 0],
                   'base_price': [33.0, 17.7, 17.7, 76.7, 76.7, 39.7, 39.7, 18.0, 84.8, 84.8, 84.8, 73.6, 73.6, 73.6, 58.6, 35.2, 87.0],
                   'comp_price': [np.nan, 16.4, 21.8, 77.0, 73.9, 37.4, 47.9, 22.4, 106.0, 104.7, 80.9, 31.7, 33.6, 74.7, 71.3, np.nan, 88.2]})

Expected = pd.DataFrame({'sku': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'agg': ['max', 'med', 'avg', 'rnk', 'max', 'max', 'min', 'med', 'rnk', 'rnk'],
                         'base_price' : [33.0, 17.7, 76.7, 39.7, 18.0, 84.8, 73.6, 58.6, 35.2, 87.0],
                         'comp_price': [np.nan, 19.10, 75.45, 37.40, 22.40, 106.00, 31.70, 71.30, np.nan, 88.20],
                         'new_price': [33.00, 19.10, 75.45, 37.40, 18.00, 84.80, 73.60, 58.60, 35.20, 88.20]})

In [12]:
df[df['agg'] == 'rnk']

,sku,agg,rank,base_price,comp_price
5,3,rnk,0,39.7,37.4
6,3,rnk,2,39.7,47.9
15,8,rnk,-1,35.2,NaN
16,9,rnk,0,87.0,88.2


In [39]:
agg_comp_price(df[df['agg'] == 'rnk'])

,sku,agg,base_price,comp_price,new_price
0,3,rnk,39.7,37.4,37.4
1,8,rnk,35.2,NaN,35.2
2,9,rnk,87.0,88.2,88.2


In [41]:
agg_comp_price(df[df['agg'] == 'rnk'])

/tmp/ipykernel_19338/2608400329.py:35: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data = x.\


,sku,agg,base_price,comp_price,new_price
0,3,rnk,39.7,37.4,37.4
1,8,rnk,35.2,NaN,35.2
2,9,rnk,87.0,88.2,88.2


In [4]:
!pwd

/home/artur/my_projects/karpov-projects/junior/competitor price


## eleastivity

In [24]:
import pandas as pd
data = pd.read_csv('COUNT_SKU_2023_10_05.csv')

In [29]:
data['sku'].value_counts()

10290    323
10230    262
10060    235
10216    129
10121     51
Name: sku, dtype: int64

In [32]:
import pandas as pd
import numpy as np

def elasticity_df(df: pd.DataFrame) -> pd.DataFrame:
    '''elastic count'''
    data = df.copy()
    def lin_reg(prices, qty):
        # Вычисление логарифмов продаж и цен
        log_sales = np.log(qty)
        log_price = np.log(prices)
        # Вычисление средних значений
        mean_log_sales = np.mean(log_sales)
        mean_log_price = np.mean(log_price)

        # Вычисление коэффициентов линейной регрессии (a и b) вручную
        b_coef = np.sum((log_price - mean_log_price) * (log_sales - 
                        mean_log_sales)) / np.sum((log_price - mean_log_price)**2)
        a_coef = mean_log_sales - b_coef * mean_log_price

        # Создание предсказанных значений
        predicted_log_sales = a_coef + b_coef * log_price

        ssr = np.sum((predicted_log_sales - mean_log_sales)**2)
        sst = np.sum((log_sales - mean_log_sales)**2)
        r2_coef = ssr / sst

        return r2_coef

    data = data.groupby(['sku'])['price', 'qty'].apply(lambda x: lin_reg(x['price'],
                                            x['qty'])).reset_index(name='elasticity')

    return data


In [33]:
elasticity_df(data)

/tmp/ipykernel_96791/733392219.py:29: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data = data.groupby(['sku'])['price', 'qty'].apply(lambda x: lin_reg(x['price'],


,sku,elasticity
0,10060,0.082813
1,10121,0.000604
2,10216,0.087294
3,10230,0.021931
4,10290,0.014512
